# 浏览器沙箱 文档

## 描述

浏览器沙箱是一个安全的浏览器执行环境，为AI Agent提供完整的浏览器自动化能力。沙箱内置了浏览器服务，支持通过VNC进行可视化操作，支持通过CDP协议进行程序化控制。

### 主要特性

- **可视化界面**: 通过VNC提供完整的桌面环境，可直接操作浏览器
- **程序化控制**: 支持CDP，自动化浏览器操作
- **文件系统集成**: 与沙箱文件系统无缝集成，支持文件上传下载（详见e2b_base.ipynb的文件系统操作）
- **安全隔离**: 在隔离环境中运行，保证安全性

### 使用场景

- 需要登录状态的网页自动化
- 数据抓取和爬虫
- 浏览器操作结果保存到沙箱文件系统
- 需要可视化调试的自动化任务


## 安装依赖

In [ ]:
%pip install e2b playwright

## 创建浏览器沙箱

In [ ]:
import os
from playwright.async_api import async_playwright
from e2b import Sandbox
from IPython.display import IFrame

# 在Agent Sandbox控制台中获取API Key以及可用域名
# 可通过环境变量预先设置，或在此处直接修改
if not os.getenv("E2B_DOMAIN"):
    os.environ["E2B_DOMAIN"] = "tencentags.com"
if not os.getenv("E2B_API_KEY"):
    os.environ["E2B_API_KEY"] = "your_api_key"

# 使用沙箱服务前，需先在控制台中创建沙箱工具，如"browser-v1"
# 创建浏览器沙箱，默认运行5分钟，使用timeout参数指定运行时间，此处保持运行3600s(1h)
sandbox = Sandbox.create(template="browser-v1", timeout=3600)
print(f"browser sandbox created: {sandbox.sandbox_id}")

## VNC 可视化界面
通过VNC可以直接操作沙箱内的浏览器，后续浏览器行为也可以通过VNC进行观察


In [ ]:
# 构建VNC访问URL，可以在浏览器中直接操作沙箱桌面
novnc_url = f"https://{sandbox.get_host(9000)}/novnc/vnc_lite.html?&path=websockify?access_token={sandbox._envd_access_token}"
print(f"vnc url: {novnc_url}")

IFrame(src=novnc_url, width="1920", height="1080")


## 基础浏览器控制


In [ ]:
# 构建CDP连接URL
cdp_url = f"https://{sandbox.get_host(9000)}/cdp"

async with async_playwright() as playwright:
    # 通过CDP协议连接到远程浏览器
    browser = await playwright.chromium.connect_over_cdp(
        cdp_url, 
        headers={"X-Access-Token": str(sandbox._envd_access_token)}
    )
    context = browser.contexts[0]
    page = context.pages[0]
    
    # 导航到指定界面
    await page.goto("https://tencent.com")
    await page.wait_for_load_state("networkidle")


## 文件上传与浏览器集成

将本地HTML文件上传到沙箱，然后在浏览器中打开


In [ ]:
html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Agent Sandbox Demo</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            padding: 50px;
            background-color: #f0f8ff;
        }
        h1 {
            color: #333;
            font-size: 3em;
        }
    </style>
</head>
<body>
    <h1>Welcome to Agent Sandbox</h1>
</body>
</html>
"""

with open("demo.html", "w", encoding="utf-8") as f:
    f.write(html_content)

with open("demo.html", "r", encoding="utf-8") as f:
    sandbox.files.write("demo.html", f)

async with async_playwright() as playwright:
    browser = await playwright.chromium.connect_over_cdp(
        cdp_url, 
        headers={"X-Access-Token": str(sandbox._envd_access_token)}
    )
    context = browser.contexts[0]
    page = context.pages[0]
    
    # 打开本地HTML文件
    file_url = "file:///home/user/demo.html"
    await page.goto(file_url)
    await page.wait_for_load_state("networkidle")
    
    print({await page.title()})


In [ ]:
# 关闭沙箱
sandbox.kill()